<a href="https://colab.research.google.com/github/smiley2911/Sephora-CF/blob/main/RM_Sephora_collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd
import numpy as np

In [ ]:
Data = pd.read_csv('/content/sephora_website_dataset.csv')
df = pd.DataFrame(Data)


In [ ]:
df.dtypes

id                          int64
brand                      object
category                   object
name                       object
size                       object
rating                    float64
number_of_reviews           int64
love                        int64
price                     float64
value_price               float64
URL                        object
MarketingFlags               bool
MarketingFlags_content     object
options                    object
details                    object
how_to_use                 object
ingredients                object
online_only               float64
exclusive                 float64
limited_edition           float64
limited_time_offer        float64
dtype: object

In [ ]:
df.columns

Index(['id', 'brand', 'category', 'name', 'size', 'rating',
       'number_of_reviews', 'love', 'price', 'value_price', 'URL',
       'MarketingFlags', 'MarketingFlags_content', 'options', 'details',
       'how_to_use', 'ingredients', 'online_only', 'exclusive',
       'limited_edition', 'limited_time_offer'],
      dtype='object')

In [ ]:
df.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,4,3002,66.0,75.0,...,True,online only,no options,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,1.0,0.0,0.0,0.0
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,4.5,76,2700,66.0,66.0,...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,no instructions,unknown,1.0,0.0,0.0,0.0
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,4.5,26,2600,180.0,180.0,...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Fresh Scent Type: Fresh Citr...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Li...,1.0,0.0,0.0,0.0
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,4.5,23,2900,120.0,120.0,...,True,online only,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,Panarea near Sicily is an an island suspended ...,no instructions,unknown,1.0,0.0,0.0,0.0
4,2218766,Acqua Di Parma,Fragrance,Colonia Miniature Set,5 x 0.16oz/5mL,3.5,2,943,72.0,80.0,...,True,online only,no options,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,1.0,0.0,0.0,0.0


In [ ]:
df.tail()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
7373,2044162,Shiseido,Moisturizers,Essential Energy Moisturizing Gel Cream,1.7 oz/ 50 mL,4.0,44,4400,48.0,48.0,...,False,0,no options,Which skin type is it good for?✔ Normal✔ Oily✔...,Suggested Usage:-Smooth over face each morning...,-Ashitaba: Supports natural mitochondrial ener...,0.0,0.0,0.0,0.0
7374,1724004,Shiseido,Moisturizers,Bio-Performance Glow Revival Cream,1.7 oz/ 50 mL,5.0,33,4900,110.0,110.0,...,False,0,no options,What it is:\nA high-performance- multibenefit ...,Suggested Usage:\n-Smooth over the face each m...,-Multi-Capisolve 1124™: Targets skin's capill...,0.0,0.0,0.0,0.0
7375,2100071,Shiseido,Face Wash & Cleansers,Clarifying Cleansing Foam,4.6 oz/ 125 mL,4.5,49,5100,36.0,36.0,...,False,0,no options,Which skin type is it good for?✔ Normal✔ Oily✔...,Suggested Usage:-Take approximately a dime-siz...,-Micro White Powder and White Clay: Remove di...,0.0,0.0,0.0,0.0
7376,153726,Shiseido,Mists & Essences,Eudermine Revitalizing Essence,no size,4.5,280,27400,60.0,60.0,...,False,0,no options,What it is: A bestselling softening lotion tha...,Suggested Usage:-Use daily- morning and night.,Water- Butylene Glycol- Alcohol Denat.- Glycer...,0.0,0.0,0.0,0.0
7377,2002830,Shiseido,Moisturizers,WASO: Color-Smart Day Moisturizer Oil-Free SPF...,1.9 oz/ 50 mL,4.5,257,8300,38.0,38.0,...,True,exclusive,no options,What it is: A tinted- mattifying moisturizer ...,Suggested Usage:-Smooth two pearl-sized drops ...,-Whole Loquat Cells: Help suppress sebum brea...,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(7378, 21)

In [ ]:
df.describe()

,id,rating,number_of_reviews,love,price,value_price,online_only,exclusive,limited_edition,limited_time_offer
count,7.378000e+03,7378.000000,7378.000000,7.378000e+03,7378.000000,7378.000000,7377.000000,7377.000000,7377.000000,7377.000000
mean,1.947067e+06,4.008674,281.047980,1.589950e+04,49.707280,51.512635,0.244137,0.259726,0.089196,0.000407
std,4.041757e+05,0.975951,869.866725,4.243198e+04,45.551533,48.014221,0.429604,0.438514,0.285046,0.020163
min,4.040000e+02,0.000000,0.000000,0.000000e+00,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,1.802650e+06,4.000000,11.000000,1.700000e+03,25.000000,25.000000,0.000000,0.000000,0.000000,0.000000
50%,2.067308e+06,4.000000,49.000000,4.900000e+03,35.000000,36.000000,0.000000,0.000000,0.000000,0.000000
75%,2.224572e+06,4.500000,215.000000,1.370000e+04,59.000000,60.000000,0.000000,1.000000,0.000000,0.000000
max,2.359685e+06,5.000000,19000.000000,1.300000e+06,549.000000,549.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7378 entries, 0 to 7377
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      7378 non-null   int64  
 1   brand                   7378 non-null   object 
 2   category                7378 non-null   object 
 3   name                    7378 non-null   object 
 4   size                    7378 non-null   object 
 5   rating                  7378 non-null   float64
 6   number_of_reviews       7378 non-null   int64  
 7   love                    7378 non-null   int64  
 8   price                   7378 non-null   float64
 9   value_price             7378 non-null   float64
 10  URL                     7378 non-null   object 
 11  MarketingFlags          7378 non-null   bool   
 12  MarketingFlags_content  7378 non-null   object 
 13  options                 7378 non-null   object 
 14  details                 7378 non-null   

In [ ]:
df.isnull()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7373,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7374,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7375,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7376,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
df.isnull().sum()

id                        0
brand                     0
category                  0
name                      0
size                      0
rating                    0
number_of_reviews         0
love                      0
price                     0
value_price               0
URL                       0
MarketingFlags            0
MarketingFlags_content    0
options                   0
details                   0
how_to_use                0
ingredients               0
online_only               1
exclusive                 1
limited_edition           1
limited_time_offer        1
dtype: int64

In [ ]:
df.rating.value_counts()

4.5    2876
4.0    2349
3.5     825
5.0     679
0.0     296
3.0     236
2.5      68
2.0      27
1.0      19
1.5       3
Name: rating, dtype: int64

In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
data = Dataset.load_from_df(df[['id', 'category', 'rating']], reader)


In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)


In [ ]:
model = SVD()
model.fit(trainset)

In [ ]:
predictions = model.test(testset)


In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9094


0.9094305779833497

In [ ]:
user_id = 1234
n = 10

In [ ]:
rated_items = df.loc[df['id'] == user_id]['category'].tolist()

In [ ]:
unrated_items = df.loc[~df['category'].isin(rated_items)]['category'].unique()

In [ ]:
predictions = [model.predict(user_id, item_id) for item_id in unrated_items]

In [ ]:
predictions.sort(key=lambda x: x.est, reverse=True)
top_n = predictions[:n]
for prediction in top_n:
    print('Item ID:', prediction.iid, 'Estimated rating:', prediction.est)


Item ID: Shaving Estimated rating: 4.522800018403258
Item ID: Cologne Estimated rating: 4.485625917289185
Item ID: Facial Peels Estimated rating: 4.338636279407298
Item ID: Aftershave Estimated rating: 4.327181003283274
Item ID: Face Oils Estimated rating: 4.322252645958976
Item ID: Exfoliators Estimated rating: 4.30696468476255
Item ID: Eye Brushes Estimated rating: 4.295155921006528
Item ID: Blush Estimated rating: 4.268988785506887
Item ID: Cheek Palettes Estimated rating: 4.267228466052365
Item ID: Body Products Estimated rating: 4.253931652365832
